This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to rotation splines](index.ipynb)

# Non-Uniform Catmull--Rom-Like Rotation Splines

> What is the best way to allow
varying intervals between sequence points in parameter
space?
>
> ---<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>, section 6: "Questions"

In the [uniform case](catmull-rom-uniform.ipynb)
we have used
[de Casteljau's algorithm with Slerp](de-casteljau.ipynb)
to create a "cubic" rotation spline.

To extend this to the non-uniform case,
we can transform the parameter $t \to \frac{t - t_i}{t_{i+1} - t_i}$
(as shown in
[the notebook about non-uniform Euclidean Bézier splines](../euclidean/bezier-non-uniform.ipynb))
for each spline segment,
which is implemented in the
[splines.quaternion.DeCasteljau](../python-module/splines.quaternion.rst#splines.quaternion.DeCasteljau) class.

In [ ]:
from splines.quaternion import DeCasteljau

Assuming the control points at the start and the end of each segment are given
(from a sequence of quaternions to be interpolated),
we'll also need a way to calculate the missing two control points.
For inspiration,
we can have a look at the
[notebook about non-uniform (Euclidean) Catmull--Rom splines](../euclidean/catmull-rom-non-uniform.ipynb#Using-Non-Uniform-Bézier-Segments)
which provides these equations:

\begin{align*}
\boldsymbol{v}_i &= \frac{
\boldsymbol{x}_{i+1} - \boldsymbol{x}_i
}{
t_{i+1} - t_i
}
\\
\boldsymbol{\dot{x}}_i
&= \frac{
(t_{i+1} - t_i) \, \boldsymbol{v}_{i-1} + (t_i - t_{i-1}) \, \boldsymbol{v}_i
}{
t_{i+1} - t_{i-1}
}
\\
\boldsymbol{\tilde{x}}_i^{(+)}
&= \boldsymbol{x}_i + \frac{(t_{i+1} - t_i) \, \boldsymbol{\dot{x}}_i}{3}
\\
\boldsymbol{\tilde{x}}_i^{(-)}
&= \boldsymbol{x}_i - \frac{(t_i - t_{i-1}) \, \boldsymbol{\dot{x}}_i}{3}
\end{align*}

With the
[relative rotation](quaternions.ipynb#Relative-Rotation-(Global-Frame-of-Reference))
$\delta_i = q_{i+1} {q_i}^{-1}$
we can try to "translate" this to quaternions
(using some vector operations in the tangent space):

\begin{align*}
\vec{\rho}_{i} &= \frac{\ln(\delta_{i})}{t_{i+1} - t_i}
\\
\vec{\omega}_i &=
\frac{
(t_{i+1} - t_i) \, \vec{\rho}_{i-1} + 
(t_i - t_{i-1}) \, \vec{\rho}_{i}
}{
t_{i+1} - t_{i-1}
}
\\
\tilde{q}_i^{(+)}
&\overset{?}{=}
\exp\left(\frac{t_{i+1} - t_i}{3} \, \vec{\omega}_i\right) \, q_i
\\
\tilde{q}_i^{(-)}
&\overset{?}{=}
\exp\left(\frac{t_i - t_{i-1}}{3} \, \vec{\omega}_i\right)^{-1} \, q_i,
\end{align*}

where $\vec{\rho}_{i}$ is the angular velocity
along the great arc from $q_i$ to $q_{i+1}$
within the parameter interval from $t_i$ to $t_{i+1}$
and
$\vec{\omega}_i$ is the angular velocity
of the Catmull--Rom-like quaternion curve
at the control point $q_i$
(which is reached at parameter value $t_i$).
Finally, $\tilde{q}_i^{(-)}$ and $\tilde{q}_i^{(+)}$
are the control quaternions before and after $q_i$, respectively.

In [ ]:
from splines.quaternion import UnitQuaternion

In [ ]:
def control_quaternions1(qs, ts):
    q_1, q0, q1 = qs
    t_1, t0, t1 = ts
    rho_in = q_1.rotation_to(q0).log_map() / (t0 - t_1)
    rho_out = q0.rotation_to(q1).log_map() / (t1 - t0)
    w0 = ((t1 - t0) * rho_in + (t0 - t_1) * rho_out) / (t1 - t_1)    
    return [
        UnitQuaternion.exp_map(-w0 * (t0 - t_1) / 3) * q0,
        UnitQuaternion.exp_map(w0 * (t1 - t0) / 3) * q0,
    ]

This is implemented in the
[splines.quaternion.DeCasteljau](../python-module/splines.quaternion.rst#splines.quaternion.DeCasteljau) class.

There is a similar (but not quite identical) way
that doesn't use the tangent space,
based on one of the approaches in
[the notebook about the uniform case](catmull-rom-uniform.ipynb):

\begin{align*}
q_{i,\text{in}} &= {\delta_{i-1}}^{\frac{t_{i+1} - t_i}{3 (t_i - t_{i-1})}}
\\
q_{i,\text{out}} &= {\delta_i}^{\frac{t_i - t_{i-1}}{3 (t_{i+1} - t_i)}}
\\
q_{i,\text{offset}} &=
\left(
\left(q_{i,\text{out}} {q_{i,\text{in}}}^{-1}\right)^\frac{1}{2} \, q_{i,\text{in}}
\right)^2
\\
\tilde{q}_i^{(+)}
&\overset{?}{=}
{q_{i,\text{offset}}}^\frac{t_{i+1} - t_i}{t_{i+1} - t_{i-1}} \, q_i
\\
\tilde{q}_i^{(-)}
&\overset{?}{=}
{q_{i,\text{offset}}}^{-\frac{t_i - t_{i-1}}{t_{i+1} - t_{i-1}}} \, q_i
\end{align*}

In [ ]:
def control_quaternions2(qs, ts):
    q_1, q0, q1 = qs
    t_1, t0, t1 = ts
    q_in = q_1.rotation_to(q0)**((t1 - t0) / (3 * (t0 - t_1)))
    q_out = q0.rotation_to(q1)**((t0 - t_1) / (3 * (t1 - t0)))
    q_offset = (q_in.rotation_to(q_out)**(1 / 2) * q_in)**2
    return [
        q_offset**(-(t0 - t_1) / (t1 - t_1)) * q0,
        q_offset**((t1 - t0) / (t1 - t_1)) * q0,
    ]

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

[splines.quaternion.canonicalized()](../python-module/splines.quaternion.rst#splines.quaternion.canonicalized)

In [ ]:
from splines.quaternion import canonicalized

In [ ]:
def create_closed_curve(rotations, grid, control_quaternion_func):
    assert len(rotations) + 1 == len(grid)
    rotations = rotations[-1:] + rotations + rotations[:2]
    # Avoid angles of more than 180 degrees (including the added rotations):
    rotations = list(canonicalized(rotations))
    first_interval = grid[1] - grid[0]
    last_interval = grid[-1] - grid[-2]
    extended_grid = [grid[0] - last_interval, *grid, grid[-1] + first_interval]
    control_points = []
    for qs, ts in zip(
            zip(rotations, rotations[1:], rotations[2:]),
            zip(extended_grid, extended_grid[1:], extended_grid[2:])):
        q_before, q_after = control_quaternion_func(qs, ts)
        control_points.extend([q_before, qs[1], qs[1], q_after])
    control_points = control_points[2:-2]
    segments = list(zip(*[iter(control_points)] * 4))
    return DeCasteljau(segments, grid)

In [ ]:
rotations = [
    angles2quat(0, 0, 180),
    angles2quat(0, 45, 90),
    angles2quat(90, 45, 0),
    angles2quat(90, 90, -90),
    angles2quat(180, 0, -180),
    angles2quat(-90, -45, 180),
]

In [ ]:
grid = np.array([0, 0.5, 2, 5, 6, 7, 10])

In [ ]:
s1 = create_closed_curve(rotations, grid, control_quaternions1)
s2 = create_closed_curve(rotations, grid, control_quaternions2)

In [ ]:
def evaluate(spline, frames=200):
    times = np.linspace(
        spline.grid[0], spline.grid[-1], frames, endpoint=False)
    return spline.evaluate(times)

for comparison, [Barry--Goldman](barry-goldman.ipynb)

In [ ]:
from splines.quaternion import BarryGoldman

In [ ]:
bg = BarryGoldman(rotations, grid)

In [ ]:
ani = animate_rotations({
    '1': evaluate(s1),
    '2': evaluate(s2),
    'Barry–Goldman': evaluate(bg),
}, figsize=(6, 2))

In [ ]:
display_animation(ani, default_mode='loop')

In [ ]:
max(max(map(abs, q.xyzw)) for q in (evaluate(s1) - evaluate(s2)))

## Parameterization

In [ ]:
rotations = [
    angles2quat(90, 0, -45),
    angles2quat(179, 0, 0),
    angles2quat(181, 0, 0),
    angles2quat(270, 0, -45),
    angles2quat(0, 90, 90),
]

In [ ]:
uniform2 = create_closed_curve(
    rotations, range(len(rotations) + 1), control_quaternions1)

[chordal parameterization](../euclidean/catmull-rom-properties.ipynb#Chordal-Parameterization)

In [ ]:
angles = np.array([
    a.rotation_to(b).angle
    for a, b in zip(rotations, rotations[1:] + rotations[:1])])
angles

In [ ]:
chordal_grid = np.concatenate([[0], np.cumsum(angles)])

In [ ]:
chordal2 = create_closed_curve(rotations, chordal_grid, control_quaternions1)

[centripetal parameterization](../euclidean/catmull-rom-properties.ipynb#Centripetal-Parameterization)

In [ ]:
centripetal_grid = np.concatenate([[0], np.cumsum(np.sqrt(angles))])

In [ ]:
centripetal2 = create_closed_curve(
    rotations, centripetal_grid, control_quaternions1)

In [ ]:
ani = animate_rotations({
    'uniform': evaluate(uniform2),
    'chordal': evaluate(chordal2),
    'centripetal': evaluate(centripetal2),
}, figsize=(6, 2))

In [ ]:
display_animation(ani, default_mode='loop')

The other method is very similar:

In [ ]:
uniform1 = create_closed_curve(
    rotations, range(len(rotations) + 1), control_quaternions2)
chordal1 = create_closed_curve(
    rotations, chordal_grid, control_quaternions2)
centripetal1 = create_closed_curve(
    rotations, centripetal_grid, control_quaternions2)

In [ ]:
max(max(map(abs, q.xyzw)) for q in (evaluate(uniform1) - evaluate(uniform2)))

In [ ]:
max(max(map(abs, q.xyzw)) for q in (evaluate(chordal1) - evaluate(chordal2)))

In [ ]:
max(max(map(abs, q.xyzw))
    for q in (evaluate(centripetal1) - evaluate(centripetal2)))